<div class="pagebreak"></div>

# Preconditions, Postconditions, Invariants, and Assertions
While this title seems long, it is ultimately the same underlying principle at work in different ways - checking if a condition holds or not. 

Looking back to the original documentation we produced for an algorithm:
* Inputs to the routine
* Outputs from the routine
* Pseudocode (steps in the routine)

We can add two additional items: _preconditions_ to the routine and _postconditions_ of the routine.

_Preconditions_ are conditions guaranteed to be true before the routine. These conditions
include such things as inputs in appropriate ranges and files initialized and ready for
input/output. _Preconditions_ are the client code’s obligations to the routine that it 
calls. For example, if we create a shape such as a circle or a square, the radius or 
side length must be a nonnegative number. As we write code, we should check that these
preconditions hold. If not, an appropriate error handling technique is needed, such as
logging the issue, returning an error code, or raising an exception.

_Postconditions_ are the conditions(items) guaranteed to be true before control returns to the caller. These conditions are the routine’s/class’s obligations to the code that uses it. For example, if a routine adds an element to a list, a postcondition is that the list is not empty. Or, more strongly, the length of a list is one more than the length of that list when the routine started.

_Invariants_ are a set of conditions(assumptions) that must always hold true during the life of a program (or, more narrowly, by the life of an object). You can consider this to be a combined precondition and postcondition - it must be valid before and after a call to a method. However, while executing a routine that changes the relevant state/attributes for the condition, the invariant may not necessarily be true. Invariants should hold during the object’s lifetime - from the end of its constructor to the end of its destructor. As an example, the radius of a circle must always be nonnegative. We also apply the term invariant to a sequence of operations. In this user, the invariant is the set of conditions that remain true during these operations.

_Assertions_ are a series of statements used during development that allows a program to check itself as it runs. _Assertions_ are sanity checks that conditions hold true. We use _assertions_ to document assumptions in the code. Primarily, we use _assertions_ as a development/debugging tool - it is possible to turn off these checks completely. Do not use assert statements for necessary checks (data validation, authorization).

## Assertion Example
Suppose we build an online stock market system and need to compute the purchase price of a stock. As part of the business rules, the company can offer incentives such as a discount on the transaction cost for special promotions such as new customer enticement.
<pre>
def compute_charge(num_shares, price_per_share, commission_charge, bonus_offer):
    amount = num_shares * price_per_share + commission_charge - bonus_offer
    return amount
</pre>
However, what happens if `bonus_offer > num_shares * price_per_share + commission_charge`? Do we owe the client money due to the client buying stocks?  In this situation, we need an assertion that the amount is positive.  We could also add another assertion that the `amount` is less than or equal to the <code>num_shares * price_per_share + commission_charge</code>. (This covers negative values passed in `bonus_offer`.)

In [ ]:
def compute_charge(num_shares, price_per_share, commission_charge, bonus_offer):
    amount = num_shares * price_per_share + commission_charge - bonus_offer
    assert 0 <= amount <= num_shares * price_per_share + commission_charge
    return amount

In [ ]:
compute_charge(100, 10.00, 7.95, 200.00)

In [ ]:
compute_charge(100, 10.00, 7.95, -200.00)  # Will raise an AssertionError as the bonus offer is negative

In [ ]:
compute_charge(100, 10.00, 7.95, 1500.00)  # raises an assertion error as amount < 0

Fixing the last condition requires a check to be added to `compute_charge` that prevents a negative. For a negative bonus offer, we could choose among several different possibilities. We could add validation for each of the function's parameters.  We could also simply not choose to perform any validation as the call came from within our program.

In [ ]:
def compute_charge(num_shares, price_per_share, commission_charge, bonus_offer):
    amount = num_shares * price_per_share + commission_charge - bonus_offer
    if amount < 0:
        amount = 0
    assert 0 <= amount <= num_shares * price_per_share + commission_charge
    return amount

In [ ]:
compute_charge(100, 10.00, 7.95, 1500.00) 

## Syntax
The assert syntax follows this format:
<pre>
    assert <i>expression</i>[, <i>message</i>]
</pre>
At a minimum, assert needs an expression that evaluates to True or False. The second portion is optional and provides an error message if the assertion fails.

## Example
Here's a slightly more involved example.

In [ ]:
class Circle:
    def __init__(self, radius):
        if radius < 0:
            raise ValueError("Radius cannot be a negative number")
        self.__radius = radius
    
    def compute_circumference(self):
        assert self.__radius >= 0, "Discovered negative radius"
        return 2 * self.__radius * 3.14159

At first glance, the assert in `compute_circumference` does not seem necessary.  However, a programmer could have manually changed a circle's radius after the circle's construction.

In [ ]:
a = Circle(2.5)
a._Circle__radius = -1.0
a.compute_circumference()  # causes an AssertionError

However, suppose a coworker changed the Circle class to add a method that allows a correction percentage that would modify a radius.  That correction percentage could be positive or negative.  The initial implementation could have been - 

In [ ]:
class Circle:
    def __init__(self, radius):
        if radius < 0:
            raise ValueError("Radius cannot be a negative number")
        self.__radius = radius
    
    def compute_circumference(self):
        assert self.__radius >= 0, "Discovered negative radius"
        return 2 * self.__radius * 3.14159
    
    def apply_correction(self, percent_change):
        self.__radius *= percent_change

In [ ]:
a = Circle(2.5)
a.apply_correction(-.01)
a.compute_circumference()

The coworker had not fully considered the potential side issues the change may have. While this is relatively simple, similar problems occur in much more complex situations. Using the `assert`, we at least have a sanity check. Our testing may or may not have found this particular situation.

In looking at the `Circle` class, we applied name mangling to highlight that developers should not directly change the values. However, we can create a more robust situation by validating the radius whenever it is changed. In this solution, we establish a getter and setter property for radius. The radius setter always validates that its value is nonnegative.

In [ ]:
class Circle:
    def __init__(self, radius):
        self.radius = radius
    
    @property
    def radius(self):
        return self.__radius
    
    @radius.setter
    def radius(self, radius):
        if radius < 0:
            raise ValueError("Radius cannot be a negative number")
        self.__radius = radius
    
    def compute_circumference(self):
        assert self.radius >= 0, "Discovered negative radius"
        return 2 * self.radius * 3.14159
    
    def apply_correction(self, percent_change):
        self.radius *= percent_change

In [ ]:
a = Circle(-3)                # raises a ValueError

In [ ]:
b = Circle(3)
b.apply_correction (-1.01)    # raises a ValueError

## Summary
Assertions are an underutilized tool in many programmers’ arsenal. By documenting assumptions, asserts can help programmers identify situations that violate those assumptions. Pay careful attention, though, when using assertions - they are not a mechanism for detecting run-time errors, as they can be globally disabled through the Python interpreter. Also, do not use asserts to implement application functionality (e.g., data validation).

## Exercises
???  may need something more complex.  given them a class file that they need to change